In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 3
setup_one_gpu(GPU)

Picking GPU 3


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets

from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch, \
                                           plot_interpolations

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  
from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from tf_lab.point_clouds.raw_gan import RawGAN

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [26]:
n_pc_samples = 2048
do_training = True
first_time_running = True
load_model = False
seed = 42

class_name = raw_input('Give me the class type.\n').lower()
syn_id = shape_net_category_to_synth_id()[class_name]
experiment_name = 'raw_pc_gan_'+ class_name +'_' + str(n_pc_samples) +  '_pts'

synthetic_data_out_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/NIPS/our_samples/'

In [5]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'

train_dir = osp.join(top_data_dir, 'OUT/models/nips/gan/raw_gan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
full_pclouds_path = osp.join(full_pclouds_path, syn_id)
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path, '.ply')
full_pclouds, full_model_ids, full_syn_ids = pio.load_crude_point_clouds(file_names=full_file_names,\
                                                                           n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )

6778 files containing complete point clouds were found.


In [6]:
full_data = PointCloudDataSet(full_pclouds)

In [10]:
reset_tf_graph()
init_lr = 0.001
gan = RawGAN(experiment_name, learning_rate=init_lr, n_output=[n_pc_samples, 3])

In [14]:
from tf_lab.point_clouds.gan import model_saver_id

train_stats = []
train_data = full_data

batch_size = 50
max_epochs = 50
noise_params = {'mu':0, 'sigma': 1}

for _ in range(max_epochs):
    checkpoint_path = osp.join(train_dir, model_saver_id)
    loss, duration = gan._single_epoch_train(train_data, batch_size, noise_params)
    epoch = int(gan.sess.run(gan.epoch.assign_add(tf.constant(1.0))))    
    print epoch, loss
    gan.saver.save(gan.sess, checkpoint_path, global_step=gan.epoch)
    train_stats.append((epoch,) + loss)

2 (1.195072669894607, 1.0942630551479482)
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/gan/raw_gan/raw_pc_gan_chair_2048_pts/models.ckpt-2 is not in all_model_checkpoint_paths. Manually adding it.
3 (1.303666779730055, 0.92459169604160163)
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/gan/raw_gan/raw_pc_gan_chair_2048_pts/models.ckpt-3 is not in all_model_checkpoint_paths. Manually adding it.
4 (1.3200043501677337, 0.93352967302004497)
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/gan/raw_gan/raw_pc_gan_chair_2048_pts/models.ckpt-4 is not in all_model_checkpoint_paths. Manually adding it.
5 (1.205949232313368, 1.1307475037044949)
INFO:tensorflow:/orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/nips/gan/raw_gan/raw_pc_gan_chair_2048_pts/models.ckpt-5 is not in all_model_checkpoint_paths. Manually adding it.
6 (0.96478396643091135, 2.0212591539930416)
INFO:tensorflow:/orions4-zfs/projects

In [24]:
from tf_lab.nips.evaluate_gan import entropy_of_occupancy_grid

for i in [1, 5, 10, 20, 30, 40, 50]:
    gan.restore_model(train_dir, i)
    n_samples = 1000
    print entropy_of_occupancy_grid_(gan.generate(n_samples, noise_params), 32)

0.0234211117783
0.0414842920605
0.0303642913352
0.0401406167227
0.0336347772219
0.0154469545613
0.0351671865931


In [32]:
# Swap axis and save
syn_data = gan.generate(n_samples, noise_params)
syn_data_rot = np.empty_like(syn_data)
for i, pc in enumerate(syn_data):
    syn_data_rot[i] = pc[:, [0, 2, 1]]
save_out = osp.join(synthetic_data_out_dir, experiment_name + '_epoch_' + str(epoch))
np.savez(save_out, syn_data_rot)